In [1]:
import cv2
import random
import numpy as np
import math
import statistics
import pickle
from PIL import Image, ImageOps
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import Counter
from scipy.spatial import distance
import warnings
warnings.simplefilter("ignore", RuntimeWarning)

In [2]:
img_read_1 = Image.open(r"Q1_image.jpg") 
img_read_1 = img_read_1.resize((256, 256))
img_gray_1 = ImageOps.grayscale(img_read_1)
img_gray_1.save("Q1_data_gray.png")
img_array_1 = np.array(img_gray_1)
temp_pad_1 = np.pad(img_array_1, pad_width=1, mode='constant', constant_values=0)
count_row_pad_img = temp_pad_1.shape[0]
count_col_pad_img = temp_pad_1.shape[1]

In [3]:
value_k = 0.05
window_row = 2
window_col = 2
#Now moving window and finding the M matrix for the window
dict_window_cord = {}
list_score_pairs = []
count_window = 0
for a in tqdm(range(1,count_row_pad_img - window_row, +1)):
    end_a = a + window_row -1
    for b in range(1,count_col_pad_img - window_col, +1):
        count_window += 1
        end_b = b + window_col -1
        dict_window_cord[count_window] = [[a, end_a],[b, end_b]]
        sum_ix2, sum_iy2, sum_ix_iy = 0, 0, 0
        for c in range(a, end_a + 1):
            for d in range(b, end_b + 1):
                val_ix = temp_pad_1[c][d-1] - temp_pad_1[c][d+1]
                val_iy = temp_pad_1[c-1][d] - temp_pad_1[c+1][d]
                sum_ix2 = sum_ix2 + (val_ix * val_ix)
                sum_iy2 = sum_iy2 + (val_iy * val_iy)
                sum_ix_iy = sum_ix_iy + (val_ix * val_iy)
                
        #Finding the score
        val_determinant = (sum_ix2 * sum_iy2) - (sum_ix_iy * sum_ix_iy)
        val_trace = sum_ix2 + sum_iy2
        val_score = val_determinant - (value_k * val_trace)
        list_score_pairs.append([count_window, val_score])

100%|████████████████████████████████████████████████████████████████████████████████| 255/255 [00:02<00:00, 94.39it/s]


In [4]:
list_score_pairs.sort(key = lambda x:x[1], reverse = True)
count_corners = 150
img_rgb_array = np.array(img_read_1)
cpy_temp_pad_1 = np.copy(temp_pad_1)
for e in range(count_corners):
    window_id = list_score_pairs[e][0]
    list_window_cord = dict_window_cord[window_id]
    cord_rows, cord_cols = list_window_cord[0], list_window_cord[1]
    for f in range(cord_rows[0], cord_rows[1]+1):
        for g in range(cord_cols[0], cord_cols[1]+1):
            cpy_temp_pad_1[f][g] = 0

In [5]:
new_im = Image.fromarray(cpy_temp_pad_1)
new_im.save("Result_Harris_corners.png")
new_im.show()